# CONUS404 Spatial Aggregation Comparison

In this notebook, we will be comparing two spatial aggregation methods to aggregate from grids to polygons. One uses [`gdptools`](https://gdptools.readthedocs.io/en/latest/index.html). The other uses conservative regional methods with [`xarray`](https://docs.xarray.dev/en/stable/index.html) and [`geopandas`](https://geopandas.org/en/stable/index.html) natively (see this [Pangeo Discourse](https://discourse.pangeo.io/t/conservative-region-aggregation-with-xarray-geopandas-and-sparse/2715) for details).

The goal of this comparision is to see how the results of the two methods compare to help judge the efficacy of one versus the other.

In [ ]:
%xmode minimal
import os
import time
import xarray as xr
import geopandas as gp
import pandas as pd
import numpy as np
import sparse

import hvplot.pandas
import hvplot.xarray
import dask
import cf_xarray

from pynhd import NLDI, WaterData
import intake
import cartopy.crs as ccrs
from shapely.geometry import Polygon

import pyproj
from gdptools import WeightGen, AggGen, UserCatData

## Open dataset from Intake Catalog

First, let's begin by loading the CONUS404 daily data.

In [ ]:
hytest_cat = intake.open_catalog("https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml")
catalog = hytest_cat['conus404-catalog']
list(catalog)

As we can see there are three different locations for the `conus404-daily` data set. The locations are (1) `-onprem-hw` meaning it is stored on the USGS Hovenweep HPC, (2) `-cloud` meaning it is store in an S3 bucket, or (3) `-osn` meaning the data is on the USGS open storage network (OSN). As the OSN is free to access from any environment, we will use that for this example, but the location can easily be changed depending on your needs.

> If you change this notebook to use the CONUS404 dataset stored on S3 (options ending in `-cloud`), you will be pulling data from a `requester-pays` S3 bucket. This means you have to set up your AWS credentials, else we won't be able to load the data. Please note that reading the `-cloud` data from S3 may incur charges if you are reading data outside of the us-west-2 region or running the notebook outside of the cloud altogether. If you would like to access one of the `-cloud` options, uncomment and run the following code cell to set up your AWS credentials. You can find more info about this AWS helper function [here](../environment_set_up/Help_AWS_Credentials.md).

In [ ]:
# uncomment the lines below to read in your AWS credentials if you want to access data from a requester-pays bucket (-cloud)
# os.environ['AWS_PROFILE'] = 'default'
# %run ../environment_set_up/Help_AWS_Credentials.ipynb

Finally, read in the daily CONUS404 data set and select the accumulated grid scale precipitation. We select the precipitation rather then all variables to keep things simple for this example, but aggregation of other variables would follow the same methodology.

In [ ]:
dataset = 'conus404-daily-osn'
conus404 = catalog[dataset].to_dask()
# Include the crs as we will need it later
conus404 = conus404[['PREC_ACC_NC', 'crs']]
conus404

## Parallelize with Dask (optional)

Some of the steps we will take are aware of parallel clustered compute environments using [`dask`](https://www.dask.org/). We can start a cluster now so that future steps take advantage
of this ability. This is an optional step, but speed ups data loading significantly, especially when accessing data from the cloud.

We have documentation on how to start a Dask Cluster in different computing environments [here](../environment_set_up/clusters.md). Uncomment the cluster start up that works for your compute environment.

In [ ]:
# %run ../environment_set_up/Start_Dask_Cluster_Nebari.ipynb
## If this notebook is not being run on Nebari/ESIP, replace the above 
## path name with a helper appropriate to your compute environment.  Examples:
# %run ../environment_set_up/Start_Dask_Cluster_Denali.ipynb
# %run ../environment_set_up/Start_Dask_Cluster_Tallgrass.ipynb
# %run ../environment_set_up/Start_Dask_Cluster_Desktop.ipynb
# %run ../environment_set_up/Start_Dask_Cluster_PangeoCHS.ipynb

## Load the feature polygons

Now that we have read in the CONUS404 data, we need to read in some polygons to aggregate the data. For this example, we will use the HUC12 basins within the Delaware River Basin. To get these HUC12 polygons, we can use [PyNHD](https://docs.hyriver.io/readme/pynhd.html) to query the Hydro Network Linked Data Index (NLDI). All we need to get the basins is a downstream river gage ID. Gage ID 01482100 is what we need as it is at the mouth of the Delaware River leading out to the Delaware Bay.

In [ ]:
%%time
# USGS gage 01482100 Delaware River at Del Mem Bridge at Wilmington De
gage_id = '01482100'
delaware_basin = NLDI().get_basins(gage_id)
# Split the Basin into its Watershed Boundary Dataset (WBD) HUC12 components
huc12_basins = WaterData('wbd12').bygeom(delaware_basin.geometry[0])

Let's plot the HUC12 basins to see how they look.

In [ ]:
huc12_basins.hvplot(
    c='huc12', title="Delaware River HUC12 basins",
    coastline='50m', geo=True,
    aspect='equal', legend=False, frame_width=300
)

An important thing to note is that all geodataframes should have a coordinate reference system (CRS). Let's check to make sure our geodataframe has a CRS.

In [ ]:
huc12_basins.crs

## Limit CONUS404 Spatial Range

With the HUC12 basins read in, we only need the CONUS404 data that spans these polygons as they are the regions we will be aggregating. So, let's limit the CONUS404 spatial range to that of the basins. This will save on memory and computation. Note doing this is mainly useful when the regions footprint is much smaller than the footprint of the gridded model.

To limit the spatial range, we first need to convert the CRS of the basins to that of CONUS404. Then extract the bounding box of the basins.

In [ ]:
huc12_basins_conus404_crs = huc12_basins.to_crs(conus404.crs.crs_wkt)
bbox = huc12_basins_conus404_crs.total_bounds
bbox

Then select the CONUS404 data within the bounding box. However, when we do this, we will extend the bounds out by 5% of their range to ensure all of our basins are within the spatially limited data. We do this as the reprojections of the CRS can cause slight distortions that make polygons on the bounds not fall fully within the data.

In [ ]:
bbox_x_range = bbox[2] - bbox[0]
bbox_y_range = bbox[3] - bbox[1]
x_range = slice(bbox[0] - bbox_x_range * 0.05,
                bbox[2] + bbox_x_range * 0.05)
y_range = slice(bbox[1] - bbox_y_range * 0.05,
                bbox[3] + bbox_y_range * 0.05)

conus404 = conus404.sel(x=x_range, y=y_range)
conus404

To make sure this worked as intended, let's plot the full basin over the extracted CONUS404 data.

In [ ]:
# Select a single timestamp for simple plotting
timestamp = '2000-5-02'
cutout = conus404.sel(time=timestamp).drop_vars(['lat', 'lon'])
# We need to write the CRS to the CONUS404 dataset and
# reproject for clean plotting with hvplot
cutout = cutout.rio.write_crs(conus404.crs.crs_wkt).rio.reproject('EPSG:4326')

cutout_plt = cutout.hvplot(
    coastline='50m', geo=True,
    aspect='equal', cmap='viridis', frame_width=300
)
huc12_plt = huc12_basins.hvplot(
    geo=True, alpha=0.3, c='r'
)
cutout_plt * huc12_plt

Looks good!

## Aggregate CONUS404 to Feature Polygons

Now that we have our gridded data and polygons, it is time to aggregate them using `gdptools` and the `geopandas` and `xarray` native method.

### gdptools Aggregation

Let's start by using the `gdptools` aggregation method.

The first step to aggregating with `gdptools` is to convert the input data to a `UserCatData` class.

In [ ]:
user_data = UserCatData(
    ds=conus404,
    proj_ds=conus404.crs.crs_wkt,
    x_coord='x',
    y_coord='y',
    t_coord='time',
    var='PREC_ACC_NC',
    f_feature=huc12_basins,
    proj_feature=huc12_basins.crs,
    id_feature='huc12',
    period=[pd.Timestamp(conus404.time.values.min()),
            pd.Timestamp(conus404.time.values.max())],
)

The `UserCatData` can then be used to generate weights for each polygon. An important thing to note is that when generating the weights we need to use an equal area projection (i.e., equal area CRS).

In [ ]:
crs_area = "EPSG:6931" # Good for northern hemisphere
# crs_area = "EPSG:5070" # Good for CONUS

# time the weight generation for later comparison
t0 = time.time()

weight_gen = WeightGen(
    user_data=user_data,
    method="dask",
    weight_gen_crs=crs_area,
)

df_gdptools_weights = weight_gen.calculate_weights()

gdptools_weights_time = time.time() - t0

df_gdptools_weights

With the weights, we can now perform the aggregation.

In [ ]:
t0 = time.time()

agg_gen = AggGen(
    user_data=user_data,
    # Use masked to ignore NaNs
    # Note that a we use mean vs sum as sum seems to ignore
    # weights even though they should be equivalent methods
    # (i.e., weighted sum = weighted mean)
    stat_method="masked_mean",
    agg_engine="dask",
    weights=df_gdptools_weights,
    agg_writer='none',
)
_, gdptools_aggregation = agg_gen.calculate_agg()

gdptools_agg_time = time.time() - t0

gdptools_aggregation

Let's make a nice plot of the aggregated HUC12 basins to make sure the aggregation worked as expected.

In [ ]:
# xarray holds the huc12s in sorted order
gdptools_huc12_basins = huc12_basins.copy().sort_values('huc12')
gdptools_huc12_basins['aggregation'] = gdptools_aggregation.sel(time=timestamp)['PREC_ACC_NC']

gdptools_plt = gdptools_huc12_basins.hvplot(
    c='aggregation', title="Accumulated Precipitation over HUC12 basins",
    coastline='50m', geo=True, cmap='viridis',
    aspect='equal', legend=False, frame_width=300
)

cutout_plt * gdptools_plt + cutout_plt

### Native Method

For the native method, we first need to extract the grid information from our CONUS404 data set. We then use it to create polygon boxes that we overlay with the basin polygons to generate weights. Finally like `gdptools`, we use the weights to aggregate via a weighted sum. 

To give a fair computational time comparison with `gdptools`, we will group all steps to generate the weights into one timed cell.

#### Create Weights

To generate the weights, we (1) extract grid information (includes extracting the `x` and `y` grid and getting their bounds), (2) use these bounds to create polygons of the grid, (3) assign the polygons to a `GeoDataFrame` with the CONUS404 dataset's CRS, (4) overlay the grid polygons and basin polygons, (5) use the overlay to get fractional area weights.

In [ ]:
%%time
t0 = time.time()
# (1) extract grid info
grid = conus404[['x', 'y']].drop_vars(['lat', 'lon']).reset_coords()
grid = grid.cf.add_bounds(['x', 'y'])


# (2) create polygons of the grid
# use a simple helper function. This way we can use xarray to parallelize.
def bounds_to_poly(x_bounds, y_bounds):
    return Polygon([
        (x_bounds[0], y_bounds[0]),
        (x_bounds[0], y_bounds[1]),
        (x_bounds[1], y_bounds[1]),
        (x_bounds[1], y_bounds[0])
    ])

# Stack the grid cells into a single stack (i.e., x-y pairs)
points = grid.stack(point=('y', 'x'))

# Apply the function to create polygons from bounds
boxes = xr.apply_ufunc(
    bounds_to_poly,
    points.x_bounds,
    points.y_bounds,
    input_core_dims=[("bounds",),  ("bounds",)],
    output_dtypes=[np.dtype('O')],
    vectorize=True
)


# (3) assign polygons to geodataframe with CRS
grid_polygons = gp.GeoDataFrame(
    data={"geometry": boxes.values, "y": boxes['y'], "x": boxes['x']},
    index=boxes.indexes["point"],
    crs=conus404.crs.crs_wkt
)


# (4) overlay the grid polygons with basin polygons
# transform both to an area preserving projection
huc12_basins_area = huc12_basins.to_crs(crs_area)
grid_polygons = grid_polygons.to_crs(crs_area)

# overlay the polygons.
overlay = grid_polygons.overlay(huc12_basins_area, keep_geom_type=True)


# (5)calculate the area fraction for each region
grid_cell_fraction = overlay.geometry.area.groupby(overlay['huc12']).transform(lambda x: x / x.sum())

# turn this into a series
multi_index = overlay.set_index(['y', 'x', 'huc12']).index
df_native_weights = pd.Series(grid_cell_fraction.values, index=multi_index)

da_native_weights_stacked = xr.DataArray(df_native_weights)

# unstack to a sparse array.
native_weights = da_native_weights_stacked.unstack(sparse=True, fill_value=0.)

native_weights_time = time.time() - t0

native_weights

Now that we have our weights, we can clearly see that this is a **sparse** matrix, with a density of ~0.0025 (i.e., only 0.25% of values are non-zero). So, maintaining it as a sparse martix is the right move for memory conservation, especially as this process scales up.

Also, this process is area conserving. We can verify this for each basin's area with a simple area calculation.

In [ ]:
# calculate areas of HUC12s from overlay and original polygons
overlay_area = overlay.geometry.area.groupby(overlay['huc12']).sum()
huc12_area = huc12_basins_area.geometry.area.groupby(huc12_basins_area['huc12']).sum()
# find the max fractional difference
(np.abs(overlay_area - huc12_area) / huc12_area).max()

Nice! This means the differences can be attributed to machine precision.

We can also verify that the cell fractions all sum up to one.

In [ ]:
grid_cell_fraction.groupby(overlay['huc12']).sum().unique()

#### Perform Aggregation

To aggregate the data, we can use [`xarray.Dataset.weighted`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.weighted.html) to do our weighted calculations. This is simple as it will take a sparse array as weights and compute the aggregation.

In [ ]:
%%time
t0 = time.time()

native_aggregation = conus404.drop_vars('crs').weighted(native_weights).sum(dim=['x', 'y']).compute()

native_agg_time = time.time() - t0

native_aggregation

Like the `gdptools` aggregation results, let's make some plots to make sure this worked as expected.

In [ ]:
# xarray holds the huc12s in sorted order
native_huc12_basins = huc12_basins.copy().sort_values('huc12')
native_huc12_basins['aggregation'] = native_aggregation.sel(time=timestamp)['PREC_ACC_NC'].data.todense()
native_plt = native_huc12_basins.hvplot(
    c='aggregation', title="Accumulated Precipitation over HUC12 basins",
    coastline='50m', geo=True, cmap='viridis',
    aspect='equal', legend=False, frame_width=300
)

cutout_plt * native_plt + cutout_plt

## Compare the Results

With both aggregation methods complete, we are now ready to compare the results. We can do this both for the final output and the intermediate weights.

### Weight Comparison

To do the weight comparison, we first need to standardize the weight outputs. This is relatively simple as we just need to convert the `gdptools` `DataFrame` weights into an `xarray.DataArray`. We can do this just like we did for the conservative method, but assigning the `x` and `y` values to the `gdptools` data frame using the given indices.

In [ ]:
# Due to the buffer region, gdptools weights index values
# are off by 3. This was found from a manual inspection
df_gdptools_weights['y'] = conus404['y'].isel(y=df_gdptools_weights['i']+3).data
df_gdptools_weights['x'] = conus404['x'].isel(x=df_gdptools_weights['j']).data
gdptool_weights = xr.DataArray(
    df_gdptools_weights.set_index(['y', 'x', 'huc12'])['wght']
).unstack(sparse=True, fill_value=0)

Now, a simple max fractional difference is the simple check for how they compare.

In [ ]:
(np.abs(gdptool_weights - native_weights) / native_weights).max()

Look at that. They are identical (up to machine precision). So, the only other thing to compare would be the time required for the computation.

In [ ]:
print(f'gtptools weights computation time: {gdptools_weights_time:0.3f} seconds')
print(f'native weights computation time: {native_weights_time:0.3f} seconds')
print(f'computation time difference: {(gdptools_weights_time - native_weights_time):0.3f} seconds')
print(f'computation time ratio: {(gdptools_weights_time / native_weights_time):0.3f}')

So, from this comparison, we can see that both methods give the same weights, but the method using `xarray` and `geopandas` natively is ~10x faster. However, this does not test how well either of the two methods scale.

### Aggregation Comparison

To do the aggregated data comparison, there is no need for any data formatting, as both `gdptools` and the native method have matching `xarray.Dataset` formats. So, let's start with the simple max fractional difference to compare.

In [ ]:
(np.abs(gdptools_aggregation - native_aggregation) / native_aggregation).max()

Well, as expected, they are nearly identical, since they had nearly identical weights.

Let's plot the fractional difference for a timestamp just to see how they compare.

In [ ]:
# xarray holds the huc12s in sorted order
native_huc12_basins = huc12_basins.copy().sort_values('huc12')
native_huc12_basins['aggregation'] = (np.abs(gdptools_aggregation - native_aggregation) / native_aggregation).sel(time=timestamp)['PREC_ACC_NC']
native_plt = native_huc12_basins.hvplot(
    c='aggregation', title="Accumulated Precipitation over HUC12 basins",
    coastline='50m', geo=True, cmap='viridis',
    aspect='equal', legend=False, frame_width=300
)

native_plt

Finally, let's compare the computational times.

In [ ]:
print(f'gtptools aggregation computation time: {gdptools_agg_time:0.3f} seconds')
print(f'native aggregation computation time: {native_agg_time:0.3f} seconds')
print(f'computation time difference: {(gdptools_agg_time - native_agg_time):0.3f} seconds')
print(f'computation time ratio: {(gdptools_agg_time / native_agg_time):0.3f}')

It looks like this step takes about the same time for both. So, let's compare the total computation time (weights and aggregation).

In [ ]:
gdptools_total_time = gdptools_weights_time+gdptools_agg_time
native_total_time = native_weights_time + native_agg_time
print(f'gtptools total computation time: {gdptools_total_time:0.3f} seconds')
print(f'native total computation time: {native_total_time:0.3f} seconds')
print(f'total computation time difference: {(gdptools_total_time - native_total_time):0.3f} seconds')
print(f'total computation time ratio: {(gdptools_total_time / native_total_time):0.3f}')

Alright, while the aggregation times are about equal, the decrease in weight creation for the native method makes it ~1.3x faster overall. Therefore, it appears that the native method would be the choice for computational speed. However, generating its weights was more complex than `gdptools`. So, that should be considered when choosing a method to implement.

## Shut down the Dask Client

If utilized, we should shut down the dask client.

In [ ]:
client.close()